# Numerical Methods for Option Pricing under the Black-Scholes Model

This notebook is part of a larger project which is used to simulate and analyze option pricing using numerical methods based on the Black-Scholes model and its extensions.

In this documentation we will cover:
- Theoretical background of the Black-Scholes model
- Derivation of the Black-Scholes Partial Differential Equation (PDE)
- Analytical solution for European options
- Numerical approaches:
    - Monte Carlo simulation
    - Finite Difference Methods
    - SDE simulation via Euler-Maruyama

## 1. Theoretical Background

### 1.1 Geometric Brownian Motion (GBM)
We assume the underlying asset price $S(t)$ follows a **Geometric Brownian Motion**:
$$
dS_t = \mu S_t dt + \sigma S_t dW_t
$$
where:
- $\mu$ is the expected return
- $\sigma$ is the volatility
- $W_t$ is a standard Brownian motion

### 1.2 Risk-Neutral Valuation
Under the **risk-neutral measure**, the drift $\mu$ becomes the risk-free rate $r$:
$$
dS_t = r S_t dt + \sigma S_t dW_t^{\mathbb{Q}}
$$
where $W_t^{\mathbb{Q}}$ is Brownian motion under the risk-neutral measure $\mathbb{Q}$.

## 2. Black-Scholes Partial Differential Equation

Using the **Itô's Lemma** and arbitrage arguments, we derive the Black-Scholes PDE for the price of a derivative $V(t, S)$:
$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0
$$
with final condition:
$$
V(T, S) = \max(S - X, 0) \quad \text{(for a European Call)}
$$

This PDE is the starting point for our numerical methods.

## 3. Analytical Black-Scholes Formula

The solution to the Black-Scholes PDE for a European call option is:
$$
C(S, t) = S N(d_1) - X e^{-r(T-t)} N(d_2)
$$
where:
$$
d_1 = \frac{\ln(S/X) + (r + \frac{\sigma^2}{2})(T-t)}{\sigma \sqrt{T-t}}, \quad d_2 = d_1 - \sigma \sqrt{T-t}
$$
and $N(\cdot)$ is the cumulative distribution function of the standard normal distribution.

## 4. Numerical Methods Overview

We will implement and compare three major classes of numerical methods:

### 4.1 Monte Carlo Simulation
- Simulates multiple paths of the underlying asset
- Averages discounted payoff
- Works well for high-dimensional problems (e.g., path-dependent options)

Monte Carlo methods estimate expectations via random sampling. For a function $f(X)$ of a random variable $X$, the expectation is approximated by:
$$
\mathbb{E}[f(X)] \approx \frac{1}{N} \sum_{i=1}^N f(X_i)
$$
where $X_i$ are i.i.d. samples from the distribution of $X$. By the **Law of Large Numbers**, this estimate converges to the true expected value as $N \to \infty$.

#### Risk-Neutral Pricing via Monte Carlo

Under the **risk-neutral measure** $\mathbb{Q}$, the price of a European derivative is the discounted expected value of the payoff:
$$
V_0 = e^{-rT} \mathbb{E}^{\mathbb{Q}}[f(S_T)]
$$
The asset price $S_t$ follows a geometric Brownian motion (GBM):
$$
dS_t = r S_t dt + \sigma S_t dW_t^{\mathbb{Q}}
$$
This has a closed-form solution:
$$
S_T = S_0 \exp\left((r - \frac{1}{2}\sigma^2)T + \sigma \sqrt{T} Z \right), \quad Z \sim \mathcal{N}(0,1)
$$
To simulate $S_T$, generate $Z_i \sim \mathcal{N}(0,1)$ and compute:
$$
\hat{V}_0 = e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^N f(S_T^{(i)})
$$
This works for both call and put options:
- Call: $f(S_T) = \max(S_T - X, 0)$
- Put: $f(S_T) = \max(X - S_T, 0)$

### 4.2 Finite Difference Methods
- Solve the Black-Scholes PDE on a grid
- Types:
  - Explicit (forward Euler in time)
  - Implicit (backward Euler)
  - Crank-Nicolson (average of explicit and implicit)
- Can handle early exercise (e.g., American options)

For a European option, the Black-Scholes PDE is:
$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0
$$
We solve this backward in time from $T$ to $0$, with terminal condition:
$$
V(S, T) = \max(S - X, 0) \quad \text{(Call)}
$$
$$\max(X - S, 0) \quad \text{for a Put}$$

#### Discretization

- Discretize $S \in [0, S_{\text{max}}]$ into $M$ grid points: $S_i = i \Delta S$
- Discretize $t \in [0, T]$ into $N$ steps: $t_n = n \Delta t$

Let $V_i^n$ be the option value at asset level $S_i$ and time $t_n$.

#### Explicit Finite Difference Scheme

- Forward Euler in time, central difference in space.
- Fully explicit, easy to implement but **conditionally stable** (requires small $\Delta t$).

Update formula:
$$
V_i^{n} = a_i V_{i-1}^{n+1} + b_i V_i^{n+1} + c_i V_{i+1}^{n+1}
$$
with coefficients:
$$
\begin{aligned}
a_i &= \frac{1}{2} \Delta t \left( \frac{\sigma^2 i^2}{2} - \frac{r i}{2} \right) \\
b_i &= 1 - \Delta t \left( \sigma^2 i^2 + r \right) \\
c_i &= \frac{1}{2} \Delta t \left( \frac{\sigma^2 i^2}{2} + \frac{r i}{2} \right)
\end{aligned}
$$

#### Implicit Method

- Backward Euler in time.
- Requires solving a tridiagonal system at each step.
- **Unconditionally stable** but lower accuracy than Crank-Nicolson.

Matrix form:
$$
A V^{n+1} = V^n
$$
where $A$ is a tridiagonal matrix built from $a_i, b_i, c_i$ coefficients as in the explicit method but evaluated at time $n+1$.

#### Crank-Nicolson Method

- Average of explicit and implicit methods.
- Second-order accurate in both time and space.
- Requires solving a tridiagonal system.

Matrix form:
$$
A V^{n+1} = B V^n
$$
where $A$ and $B$ are tridiagonal matrices constructed from weighted combinations of the explicit and implicit coefficients.

#### American options

For an **American put option**, early exercise is possible, and the value must satisfy:
$$
V(S, t) \geq X - S \quad \text{(no arbitrage)}
$$
and the option holder exercises early if:
$$
V(S, t) = X - S
$$
This leads to a **free boundary problem** where we must simultaneously solve the PDE **and** identify the optimal exercise boundary.

#### Linear Complementarity Problem (LCP)

When using finite differences (e.g., implicit or Crank-Nicolson), the time-stepping results in a linear system:
$$
A V^{n+1} = V^n
$$
For American options, we instead solve an LCP:
$$
A V \geq b, \quad V \geq f, \quad (A V - b)^T (V - f) = 0
$$
Where:
- $f$ is the early exercise payoff (e.g., $X - S$ for a put)
- $V$ is the option value at a given timestep
- $b$ comes from the previous timestep values

#### Successive Over-Relaxation (SOR)

SOR is an iterative method to solve $Ax = b$. For each component:
$$
x_i^{(k+1)} = (1 - \omega) x_i^{(k)} + \frac{\omega}{a_{ii}}\left(b_i - \sum_{j < i} a_{ij} x_j^{(k+1)} - \sum_{j > i} a_{ij} x_j^{(k)}\right)
$$
- $\omega \in (1, 2)$ is the relaxation factor
- Accelerates convergence when optimally chosen

#### Projected SOR Algorithm

To handle the early exercise constraint $V \geq f$, we **project the solution** at each step:

**Step-by-step PSOR update:**
1. Compute standard SOR update $\tilde{V}_i^{(k+1)}$
2. Project: $V_i^{(k+1)} = \max(f_i, \tilde{V}_i^{(k+1)})$

This guarantees that the solution always satisfies the early exercise constraint.

**Stopping criterion:** Stop when:
$$
\|V^{(k+1)} - V^{(k)}\| < \text{tolerance}
$$

### 4.3 SDE Simulation (Euler-Maruyama)
- Numerically solves the SDE governing asset dynamics
- Useful for simulating price paths and validating GBM assumptions

#### SDE Simulation

To simulate paths of $S_t$, we discretize the time interval $[0, T]$ into $N$ steps:
$$
\Delta t = \frac{T}{N}
$$
and approximate the Brownian motion increment as:
$$
\Delta W_t = \sqrt{\Delta t} \cdot Z, \quad Z \sim \mathcal{N}(0, 1)
$$

#### Euler-Maruyama Method

This is the simplest numerical method for simulating an SDE:
$$
S_{t+\Delta t} = S_t + r S_t \Delta t + \sigma S_t \Delta W_t
$$
This corresponds to the discretized form of GBM. It has strong order of convergence $0.5$ and is sufficient for many financial simulations.

#### Milstein Method

The Milstein method improves accuracy by including a correction term:
$$
S_{t+\Delta t} = S_t + r S_t \Delta t + \sigma S_t \Delta W_t + \frac{1}{2} \sigma^2 S_t ( (\Delta W_t)^2 - \Delta t )
$$
This has strong order of convergence $1.0$, which improves simulation quality, especially for path-dependent options or Greeks.

#### Distribution of Final Asset Price

Because the solution to GBM is known analytically:
$$
S_T \sim \text{LogNormal}(\log S_0 + (r - \frac{1}{2}\sigma^2)T, \sigma^2 T)
$$
We can compare the histogram of simulated $S_T$ values with the PDF of this log-normal distribution:
$$
f_{S_T}(x) = \frac{1}{x \sigma \sqrt{2\pi T}} \exp\left(-\frac{(\ln x - \mu)^2}{2\sigma^2 T}\right)
$$
where $\mu = \ln S_0 + (r - \frac{1}{2}\sigma^2)T$.

This is a useful diagnostic tool to verify the quality of your simulation.

## 5. Extentions of B-S model

### 5.1 Heston model
The Heston model assumes the asset price $S_t$ and its variance $v_t$ evolve as:
$$
\begin{aligned}
dS_t &= \mu S_t dt + \sqrt{v_t} S_t dW_t^S \\
dv_t &= \kappa(\theta - v_t) dt + \xi \sqrt{v_t} dW_t^v
\end{aligned}
$$
with correlation:
$$
dW_t^S dW_t^v = \rho dt
$$
### Parameters:
- $\mu$: drift
- $\kappa$: rate of mean reversion
- $\theta$: long-run variance
- $\xi$: volatility of volatility
- $\rho$: correlation between asset and volatility

This model captures **volatility smiles** and **stochastic skew** naturally.

### 5.2 Merton Jump-Diffusion Model

In the Merton model, the asset price follows:
$$
dS_t = (\mu - \lambda k) S_t dt + \sigma S_t dW_t + S_{t-} dJ_t
$$
- $J_t$ is a **Poisson jump process** with intensity $\lambda$
- Jumps follow: $\log(Y) \sim \mathcal{N}(\mu_J, \sigma_J^2)$
- $k = \mathbb{E}[Y - 1]$ is the expected relative jump size

The jump term introduces **discontinuous paths**, useful for modeling earnings shocks or crashes.